 # **Progetto ICON**

# **Habitable exoplanets classification**



Descrizione del progetto

In [ ]:
# Import libraries
import imp
import pandas as pd
import numpy as np


from sklearn.impute import SimpleImputer

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_validate
from imblearn.over_sampling import SMOTE, ADASYN

import seaborn as sns
import matplotlib.pyplot as plt

##**Load Data**

In [ ]:
planets = pd.read_csv("/content/PHL-EC.csv")
planets.head()

,P. Name,P. Name Kepler,P. Name KOI,P. Zone Class,P. Mass Class,P. Composition Class,P. Atmosphere Class,P. Habitable Class,P. Min Mass (EU),P. Mass (EU),...,P. Int ESI,P. Surf ESI,P. ESI,S. HabCat,P. Habitable,P. Hab Moon,P. Confirmed,P. Disc. Method,P. Disc. Year,Unnamed: 68
0,1RXS 1609 b,NaN,NaN,Cold,Jovian,gas,hydrogen-rich,non-habitable,NaN,4451.16,...,0,0,0.05,0,0,0,1,Imaging,2008,NaN
1,1SWASP J1407 b,NaN,NaN,Cold,Jovian,gas,hydrogen-rich,non-habitable,6358.80,6358.80,...,0,0,0.07,0,0,0,1,Primary Transit,2012,NaN
2,2M 0103-55(AB) b,NaN,NaN,Cold,Jovian,gas,hydrogen-rich,non-habitable,4133.22,4133.22,...,0,0,0.06,0,0,0,1,Imaging,2013,NaN
3,2M 0122-24 b,NaN,NaN,Cold,Jovian,gas,hydrogen-rich,non-habitable,NaN,6358.80,...,0,0,0.08,0,0,0,1,Imaging,2013,NaN
4,2M 0219-39 b,NaN,NaN,Cold,Jovian,gas,hydrogen-rich,non-habitable,NaN,4419.37,...,0,0,0.06,0,0,0,1,Imaging,2015,NaN


##**Data Preparation**

Elimino le tipologie di pianeti per le quali abbiamo pochi esempi

In [ ]:
target_count = planets['P. Habitable Class'].value_counts()
target_count

non-habitable        3820
mesoplanet             31
psychroplanet          18
thermoplanet            3
hypopsychroplanet       3
Name: P. Habitable Class, dtype: int64

In [ ]:
indexNames = planets[planets['P. Habitable Class'] == 'thermoplanet'].index
planets.drop(indexNames, inplace= True)


indexNames = planets[planets['P. Habitable Class'] == 'hypopsychroplanet'].index
planets.drop(indexNames, inplace= True)

In [ ]:
target_count = planets['P. Habitable Class'].value_counts()
target_count

non-habitable    3820
mesoplanet         31
psychroplanet      18
Name: P. Habitable Class, dtype: int64

In [ ]:
planets

,P. Name,P. Name Kepler,P. Name KOI,P. Zone Class,P. Mass Class,P. Composition Class,P. Atmosphere Class,P. Habitable Class,P. Min Mass (EU),P. Mass (EU),...,P. Int ESI,P. Surf ESI,P. ESI,S. HabCat,P. Habitable,P. Hab Moon,P. Confirmed,P. Disc. Method,P. Disc. Year,Unnamed: 68
0,1RXS 1609 b,NaN,NaN,Cold,Jovian,gas,hydrogen-rich,non-habitable,NaN,4451.16,...,0,0,0.05,0,0,0,1,Imaging,2008,NaN
1,1SWASP J1407 b,NaN,NaN,Cold,Jovian,gas,hydrogen-rich,non-habitable,6358.80,6358.80,...,0,0,0.07,0,0,0,1,Primary Transit,2012,NaN
2,2M 0103-55(AB) b,NaN,NaN,Cold,Jovian,gas,hydrogen-rich,non-habitable,4133.22,4133.22,...,0,0,0.06,0,0,0,1,Imaging,2013,NaN
3,2M 0122-24 b,NaN,NaN,Cold,Jovian,gas,hydrogen-rich,non-habitable,NaN,6358.80,...,0,0,0.08,0,0,0,1,Imaging,2013,NaN
4,2M 0219-39 b,NaN,NaN,Cold,Jovian,gas,hydrogen-rich,non-habitable,NaN,4419.37,...,0,0,0.06,0,0,0,1,Imaging,2015,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3870,YBP1194 b,NaN,NaN,Hot,Jovian,gas,metals-rich,non-habitable,108.10,108.10,...,0,0,0.16,0,0,0,1,Radial Velocity,2014,NaN
3871,YBP1514 b,NaN,NaN,Hot,Jovian,gas,metals-rich,non-habitable,127.18,127.18,...,0,0,0.15,0,0,0,1,Radial Velocity,2014,NaN
3872,YZ Cet b,NaN,NaN,Hot,Terran,rocky-iron,metals-rich,non-habitable,0.76,0.76,...,0,0,0.43,0,0,0,1,Radial Velocity,2017,NaN
3873,YZ Cet c,NaN,NaN,Hot,Terran,rocky-iron,metals-rich,non-habitable,0.99,0.99,...,0,0,0.53,0,0,0,1,Radial Velocity,2017,NaN


In [ ]:
planets.reset_index(inplace = True)

Rimuovo alcune feauture segiendo alcune tecniche di feauture selection

count = valori nulli in una colonna specifica(in una feature)
lenplanet( planet) = numero di righe totali(tot pianeti)

count/len(planet) = ci dice in percentuale quanti valori ci saranno in quella colonna

In [ ]:
def remove_missing(feauture):
    count = len(planets[planets[feauture].isnull()])
    if count/len(planets) > 0.2:
        return 1
    else:
        return 0
    
missing_values = [x for x in planets.columns if remove_missing(x)]
planets = planets.drop(missing_values, axis=1)

In [ ]:
cols_to_drop = ['S. Constellation', 'S. Type', 'P. Int ESI', 
                'P. Surf ESI', 'P. Disc. Method', 'P. Disc. Year','P. Hab Moon', 'P. SFlux Min (EU)', 'P. SFlux Max (EU)',
                'P. Teq Min (K)','P. Teq Max (K)']

planets = planets.drop(cols_to_drop, axis=1)

Voglio rimuovere anche in base alla correlazione con l'attributo habitability class ma per usare la correlazione devo: 

*    riempire il dataset con i dati mancati
*    traformare prima la feature habitable class (categorica) in feauture numerica




riempio tramite l'inputer



In [ ]:
planets.head()

,index,P. Name,P. Zone Class,P. Mass Class,P. Composition Class,P. Atmosphere Class,P. Habitable Class,P. Mass (EU),P. Radius (EU),P. Density (EU),...,S. Hab Zone Min (AU),S. Hab Zone Max (AU),P. HZD,P. HZC,P. HZA,P. HZI,P. ESI,S. HabCat,P. Habitable,P. Confirmed
0,0,1RXS 1609 b,Cold,Jovian,gas,hydrogen-rich,non-habitable,4451.16,19.04,0.64,...,0.540,1.362,800.07,23.51,85.62,0.00,0.05,0,0,1
1,1,1SWASP J1407 b,Cold,Jovian,gas,hydrogen-rich,non-habitable,6358.80,10.94,4.86,...,0.461,1.143,9.07,15.30,45.41,0.02,0.07,0,0,1
2,2,2M 0103-55(AB) b,Cold,Jovian,gas,hydrogen-rich,non-habitable,4133.22,11.40,2.79,...,0.136,0.347,793.67,12.57,107.44,0.00,0.06,0,0,1
3,3,2M 0122-24 b,Cold,Jovian,gas,hydrogen-rich,non-habitable,6358.80,11.20,4.53,...,0.136,0.347,490.45,15.72,119.46,0.00,0.08,0,0,1
4,4,2M 0219-39 b,Cold,Jovian,gas,hydrogen-rich,non-habitable,4419.37,16.13,1.05,...,0.062,0.165,3028.82,19.46,133.25,0.00,0.06,0,0,1


In [ ]:
planets.isnull().sum()

index                          0
P. Name                        0
P. Zone Class                 46
P. Mass Class                  6
P. Composition Class          41
P. Atmosphere Class           85
P. Habitable Class             0
P. Mass (EU)                  33
P. Radius (EU)                12
P. Density (EU)               41
P. Gravity (EU)               41
P. Esc Vel (EU)               41
P. SFlux Mean (EU)             0
P. Teq Mean (K)               46
P. Surf Press (EU)            41
P. Mag                        56
P. Appar Size (deg)           12
P. Period (days)             150
P. Sem Major Axis (AU)        35
P. Eccentricity                0
P. Mean Distance (AU)         35
P. Omega (deg)                 0
S. Name                        0
S. Mass (SU)                  47
S. Radius (SU)               112
S. Teff (K)                   99
S. Luminosity (SU)            28
S. RA (hrs)                    0
S. DEC (deg)                   0
S. Mag from Planet            46
S. Size fr

In [ ]:
numeric_values = planets[planets._get_numeric_data().columns]


imputer = SimpleImputer(missing_values = np.NaN, strategy = 'mean') #di default strategy ='mean'
numeric_values = pd.DataFrame(imputer.fit_transform(numeric_values), columns=numeric_values.columns)
numeric_values.to_csv('Imputed Data.csv', index='rowid')

In [ ]:
imputed_numerics = pd.read_csv('/content/Imputed Data.csv')

In [ ]:
for i in planets.columns:
    if i not in imputed_numerics.columns:
        pass
    else:
        planets[i] = imputed_numerics[i]

trasformo la feauture categorica

In [ ]:
#Convert string values of origin column to numerical values
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(planets['P. Habitable Class'])

# finding the unique classes
print(list(label_encoder.classes_))
print()

# values after transforming the categorical column.
print(label_encoder.transform(planets['P. Habitable Class']))

planets['P. Habitable Class'] = label_encoder.transform(planets['P. Habitable Class'])

In [ ]:
cor = planets.corr('spearman')
cor.head()

In [ ]:
sns.heatmap(cor, annot = False)

In [ ]:
threshold = 0
cor_h = cor['P. Habitable Class'].sort_values(ascending=False)
result = cor_h.tail(20)
result

Possiamo notare come il dataset sia molto sblianciato

In [ ]:
target_count = planets['P. Habitable Class'].value_counts()
target_count

non-habitable    3820
mesoplanet         31
psychroplanet      18
Name: P. Habitable Class, dtype: int64

In [ ]:
target_count = planets['P. Habitable Class'].value_counts()
print(f'non-habitable: {target_count[0]}')
print(f'mesoplanet: {target_count[1]}')
print(f'psychroplanet : {target_count[2]}')
print(f'Percentage of Majority Class: {round(target_count[0] / sum(target_count), 4)*100}')
print(f'Percentage of Minority Class: {round(target_count[1] / sum(target_count), 4)*100}')
print(f'Percentage of Minority Class: {round(target_count[2] / sum(target_count), 4)*100}')

non-habitable: 3820
mesoplanet: 31
psychroplanet : 18
Percentage of Majority Class: 98.72999999999999
Percentage of Minority Class: 0.8
Percentage of Minority Class: 0.47000000000000003


Divido il dataset 

In [ ]:
#Convert string values of origin column to numerical values
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(planets['P. Habitable Class'])

# finding the unique classes
print(list(label_encoder.classes_))
print()

# values after transforming the categorical column.
print(label_encoder.transform(planets['P. Habitable Class']))

planets['P. Habitable Class'] = label_encoder.transform(planets['P. Habitable Class'])

['mesoplanet', 'non-habitable', 'psychroplanet']

[1 1 1 ... 1 1 1]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(planets.iloc[:, :-1], planets.loc[:, 'P. Habitable Class'], test_size=0.25,random_state=76)
X_train

,index,P. Name,P. Zone Class,P. Mass Class,P. Composition Class,P. Atmosphere Class,P. Habitable Class,P. Mass (EU),P. Radius (EU),P. Density (EU),...,S. No. Planets HZ,S. Hab Zone Min (AU),S. Hab Zone Max (AU),P. HZD,P. HZC,P. HZA,P. HZI,P. ESI,S. HabCat,P. Habitable
2596,2598.0,Kepler-840 b,Hot,Jovian,NaN,NaN,1,579.403626,17.06,1.52553,...,0.0,0.828,1.944,-2.42,3.72877,4.429553,0.216089,0.05,0.0,0.0
2637,2639.0,Kepler-881 b,Hot,Superterran,rocky-iron,metals-rich,1,6.560000,1.78,1.16000,...,0.0,0.861,2.019,-2.39,-0.14000,-0.650000,0.290000,0.27,0.0,0.0
3000,3002.0,Kepler-1225 b,Hot,Superterran,rocky-iron,metals-rich,1,7.040000,1.81,1.18000,...,0.0,0.987,2.304,-2.38,-0.14000,-0.570000,0.290000,0.27,0.0,0.0
2386,2388.0,Kepler-645 b,Hot,Superterran,rocky-iron,metals-rich,1,12.420000,2.09,1.35000,...,0.0,1.011,2.360,-2.43,-0.13000,-0.480000,0.290000,0.25,0.0,0.0
3161,3163.0,Kepler-1375 b,Hot,Superterran,rocky-iron,metals-rich,1,5.960000,1.74,1.14000,...,0.0,1.702,3.940,-2.48,-0.15000,-0.880000,0.280000,0.27,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,2092.0,Kepler-395 b,Hot,Terran,rocky-iron,metals-rich,1,0.960000,1.03,0.88000,...,0.0,0.246,0.613,-2.00,-0.17000,-0.950000,0.310000,0.35,0.0,0.0
2330,2332.0,Kepler-596 b,Hot,Neptunian,gas,metals-rich,1,8.210000,3.46,0.20000,...,0.0,1.131,2.636,-2.29,2.71000,-0.670000,0.220000,0.21,0.0,0.0
2432,2434.0,Kepler-690 b,Hot,Neptunian,gas,metals-rich,1,41.910000,5.95,0.20000,...,0.0,0.904,2.123,-2.35,3.91000,-0.100000,0.180000,0.14,0.0,0.0
1415,1416.0,Kepler-124 d,Hot,Terran,rocky-iron,metals-rich,1,1.230000,1.11,0.90000,...,0.0,0.373,0.903,-1.77,-0.17000,-0.750000,0.340000,0.45,0.0,0.0


effettuo oversampling

In [ ]:
X_smoted, y_smoted = SMOTE(sampling_strategy='minority', random_state=76).fit_sample(X_train, y_train)

AttributeError: ignored

In [ ]:
X_smoted.shape

In [ ]:
# observe that data has been balanced
pd.Series(y_smoted).value_counts().plot.bar()